# Airbnb princing

In Airbnb the pricing should be set by the owner of the property. The suggestion is that the rate should be based on the monthly mortgage. 

In [2]:
import pandas as pd
import numpy as np

raw_data = pd.read_csv('data/Ames_Housing_Price_Data.csv', index_col=0)

rental_df = raw_data[['PID','SalePrice','YrSold']]
rental_df.sample(10)

,PID,SalePrice,YrSold
471,907420080,189000,2008
614,528364110,290000,2007
329,535327230,100000,2008
187,527401160,136000,2009
475,907262050,237000,2010
57,923228420,81000,2008
906,903232030,81300,2008
687,916326010,228000,2006
573,907275150,275000,2007
725,903232090,162900,2008


Group by OverallQual and get the mean and median SalePrice for each category

In [3]:
mortgage_df = pd.read_csv('data/MortageRates.csv')
mortgage_df['Average_30Year_Rate']= mortgage_df['Average_30Year_Rate'].str.replace('%','').astype(float)
mortgage_df.sample(10)

,Year,Average_30Year_Rate
44,2016,3.65
25,1997,7.60
36,2008,6.03
20,1992,8.39
46,2018,4.54
29,2001,6.97
26,1998,6.94
2,1974,9.19
8,1980,13.74
41,2013,3.98


In [4]:
rental_df = pd.merge(
    rental_df,
    mortgage_df,
    how="left",
    left_on='YrSold',
    right_on='Year',
    sort=True,
    suffixes=("_x", "_y"),
    copy=False).drop(columns = ['Year'])

rental_df.sample(10)

,PID,SalePrice,YrSold,Average_30Year_Rate
946,527455030,142500,2007,6.34
262,902329090,157000,2006,6.41
546,902408080,159500,2007,6.34
590,907200270,134900,2007,6.34
2053,535375010,99900,2009,5.04
1665,531451020,193000,2008,6.03
1431,903481100,82375,2008,6.03
720,907200250,160000,2007,6.34
464,909280070,218000,2006,6.41
2444,532354090,148000,2010,4.69


In [5]:
rental_df['YrSold'].value_counts()

2007    600
2009    585
2008    564
2006    516
2010    315
Name: YrSold, dtype: int64

According to Forbes, 2006 downpayment was 20%

Calculate monthly mortgage payment, not including taxes and insurance, using the following equation:

M = P [ i(1 + i)^n ] / [ (1 + i)^n – 1]

P = 0.8*SalePrice <- principal loan amount

i = monthly interest rate

n = 30*12 = 360

In [14]:
P = 0.8*rental_df['SalePrice']
i = rental_df['Average_30Year_Rate'].map(lambda x: x/(100*12))
n = 360
rental_df['MMortgage'] = P*((i*(1+i)**n))/(((1+i)**n)-1)
rental_df.sample(10)

rental_df['MMor_utilities'] = rental_df['MMortgage'] + 215 #adding average monthly utilities
rental_df['day_rate'] = (rental_df['MMor_utilities']/30).round(2)

rental_df.sample(10)

,PID,SalePrice,YrSold,Average_30Year_Rate,MMortgage,day_rate,MMor_utilities
2543,916386060,174000,2010,4.69,721.107401,31.20,936.107401
317,909281010,239000,2006,6.41,1197.219490,47.07,1412.219490
453,908178050,229000,2006,6.41,1147.126624,45.40,1362.126624
1025,914460020,215000,2007,6.34,1069.121965,42.80,1284.121965
2033,905475500,84000,2009,5.04,362.388707,19.25,577.388707
1803,910205020,78000,2009,5.04,336.503800,18.38,551.503800
602,532353120,120750,2007,6.34,600.448732,27.18,815.448732
175,907285100,237000,2006,6.41,1187.200917,46.74,1402.200917
859,528181070,209500,2007,6.34,1041.772334,41.89,1256.772334
1166,533234020,223000,2008,6.03,1073.041465,42.93,1288.041465


monthly utilities sourcehttp://www.spacesimply.com/article.aspx?id=20&title=The+Price+We+Pay+to+Live+in+Iowa+(Infographic)